# Determine tanker transfer statistics
- Investigate ECY transfers for vessels recorded by AIS MMSI. 
- This notebook is modified from `https://github.com/MIDOSS/analysis-rachael/blob/main/notebooks/monte_carlo_QAQC/QAQC_Tanker_AISlengths_DOEcapacities.ipynb` to focus on ECY transfers

---
Use `analysis-rachael/env/monte_carlo.yaml` to create an environment for this notebook:
```
 conda env create -f [analysis-rachael/env/monte_carlo.yaml]
```
or, to activate this environment, use
```
 conda activate monte-carlo
```
To deactivate an active environment, use
```
 conda deactivate
```
---  

In [3]:
# import necessary packages
import os
import yaml
import pandas
import numpy
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import FormatStrFormatter
import geopandas as gpd
from pathlib import Path
# import function for querying ECY dataframes
from dask import delayed
from monte_carlo_utils import get_ECY_df, get_bin

# set file location and name
ecy_xls_path = Path(
    '/data/MIDOSS/spreadsheets/MuellerTrans4-30-20.xlsx'
)
fac_xls_path = Path(
    '/data/MIDOSS/marine_transport_data/Oil_Transfer_Facilities.xlsx'
)
yaml_out_dir = Path(
    '/data/MIDOSS/marine_transport_data/'
)

# import facility locations
facWA = pandas.read_excel(
    fac_xls_path,
    sheet_name = 'Washington',
    usecols="B,D,J,K"
)

# import facility locations
facCAD = pandas.read_excel(
    fac_xls_path,
    sheet_name = 'British Columbia',
    usecols="A",
    nrows=10
)

mpl.rc('font', size=11)
# some of the following may be repetetive but can also be set relative to the font value above 
#    (eg "xx-small, x-small,small, medium, large, x-large, xx-large, larger, or smaller"; see link above for details)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
mpl.rc('legend', fontsize=12)
mpl.rc('axes', titlesize=16)
mpl.rc('axes', labelsize=12)
mpl.rc('figure', titlesize=16)
mpl.rc('text', usetex=False)
mpl.rc('font', family='sans-serif', weight='normal', style='normal')

# Import Dept. of Ecology transfer data
ECYdf = get_ECY_df(
    ecy_xls_path, 
    fac_xls_path,
    group = 'no'
)

# Create list of names for tanker size classes evaluated
size_classes = [
    "Smallest Tanker (< 60 m)",
    "Small Tanker (61 - 150 m)", 
    "Handymax (151-200 m)", 
    "Aframax (201-250 m)", 
    "Suezmax (251-300 m)" 
]

SyntaxError: invalid syntax (4257113732.py, line 13)

#### Conversion factors

In [ ]:
gal2liter = 3.78541

#### Initalizate dictionaries

In [ ]:
# All AIS ship track data sorted by vessel type. 
allTracks={} 
# MMSI numbers and names by tanker size classes
mmsi_names = {}
# Number of MMSI used in each size class to evaluate cargo capacity using the ECY database
N_MMSI={}
# Percent of vessels in size class used in ECY evaluation
percent_total={}
# Tanker volume transfers
volume={}
imports={}
exports={}
one_way = {}
two_way = {}
# Probability of full tanker for hanymax (full, half-full, total)
handyVoyages={}
# Handymax oil cargo capacities and probability of full cargo
capacity = {}

In [ ]:
fac_xls_path = Path(
    '/data/MIDOSS/marine_transport_data/Oil_Transfer_Facilities.xlsx'
)
fac_xls_path

In [ ]:
def get_ecy_tanker_byvessel(vessels,ecy_xls_path,fac_xls_path):
    """
        Inputs:
            - vessels [list]: List of vessel names, e.g.["AMERICAN FREEDOM","PELICAN STATE"]
            - ecy_xls_path [path]: Location and name of ECY data spreadsheet
            - fac_xls_path [path]: Location and name of facilities transfer spreadsheet
        Outputs:
            - cargo_transfers [dataframe]: 2018 cargo transfers to/from the vessels and 
               the marine terminals used in this study, in liters.  Transfers are grouped by AntID
    """
    # conversion factor
    gal2liter = 3.78541
    # load dept. of ecology data
    ECYdf = get_ECY_df(
        ecy_xls_path, 
        fac_xls_path,
        group = 'no'
    )
    # extract tanker cargo transfers
    if isinstance(vessels, list):
        cargo_transfers = ECYdf.loc[
            (ECYdf.TransferType == 'Cargo') &
            (ECYdf.Deliverer.isin(vessels) |
             ECYdf.Receiver.isin(vessels)),
            ['TransferQtyInGallon', 'Deliverer','Receiver','StartDateTime','AntID']
        ].groupby('AntID').agg(
            {'TransferQtyInGallon':'sum',
             'Deliverer':'first', 
             'Receiver':'first',
             'StartDateTime':'first'}
            ).sort_values(by='TransferQtyInGallon',ascending=False)
    else: # if a string
         cargo_transfers = ECYdf.loc[
            (ECYdf.TransferType == 'Cargo') &
            (ECYdf.Deliverer.str.contains(vessels) |
             ECYdf.Receiver.str.contains(vessels)),
            ['TransferQtyInGallon', 'Deliverer','Receiver','StartDateTime','AntID']
        ].groupby('AntID').agg(
            {'TransferQtyInGallon':'sum',
             'Deliverer':'first', 
             'Receiver':'first',
             'StartDateTime':'first'}
            ).sort_values(by='TransferQtyInGallon',ascending=False)
    # convert to liters
    cargo_transfers['TransferQtyInGallon'] = gal2liter*cargo_transfers['TransferQtyInGallon']
    cargo_transfers=cargo_transfers.rename(
        columns={"TransferQtyInGallon":"TransferQtyInLiters"}
    ).reset_index()

    return cargo_transfers


In [ ]:
def get_ecy_byVesselType(type_description,ecy_xls_path,fac_xls_path):
    """
        Inputs:
            - type_description [list]: List of vessel type, e.g.['TANK SHIP']
            - ecy_xls_path [path]: Location and name of ECY data spreadsheet
            - fac_xls_path [path]: Location and name of facilities transfer spreadsheet
        Outputs:
            - cargo_transfers [dataframe]: 2018 cargo transfers to/from the vessels and 
               the marine terminals used in this study, in liters.  Transfers are grouped by AntID
    """
    # conversion factor
    gal2liter = 3.78541
    # load dept. of ecology data
    ECYdf = get_ECY_df(
        ecy_xls_path, 
        fac_xls_path,
        group = 'no'
    )
    # extract tanker cargo transfers
    cargo_transfers = ECYdf.loc[
        (ECYdf.TransferType == 'Cargo') &
        (ECYdf.DelivererTypeDescription.isin(type_description) |
         ECYdf.ReceiverTypeDescription.isin(type_description)),
        ['TransferQtyInGallon', 'StartDateTime','AntID']
    ].groupby('AntID').agg(
        {'TransferQtyInGallon':'sum','StartDateTime':'first'}
        ).sort_values(by='TransferQtyInGallon',ascending=False)

    cargo_transfers['TransferQtyInGallon'] = gal2liter*cargo_transfers['TransferQtyInGallon']
    cargo_transfers=cargo_transfers.rename(
        columns={"TransferQtyInGallon":"TransferQtyInLiters"}
    ).reset_index()
    return cargo_transfers

In [ ]:
def bins_labels(bins, **kwargs):
    """
    Labels x-axis of histogram with labels in center of bar
    Inputs:
        - bins [vector]: e.g. bins = numpy.arange(0,1.7e8,1e7)
    """
    bin_w = (max(bins) - min(bins)) / (len(bins) - 1)
    plt.xticks(numpy.arange(min(bins)+bin_w/2, max(bins)+2*bin_w/2, bin_w), bins, **kwargs, rotation=90)
    plt.xlim(bins[0], bins[-1])

In [ ]:
# This version replaces append with concat.  Needs debugging.  Gets hung up with Smallest.
def split_ecy_transfers(ecy_df):
    """
    split dataframe of ECY transfers into two-way transfers (import and export) and one-way transfers
    
    Updates:
    - [8/3/23] 
        - Addressed python warning that .append() is being retired. Changed .append() to pandas.concat()
        - inroduced "try" "except" for creating dataframe
    """
    one_way=pandas.DataFrame({})
    two_way=pandas.DataFrame({})
    count = 0
    idx_taken = 0
    # order transfers by time
    ecy_df = ecy_df.sort_values(by='StartDateTime').reset_index(drop=True)
    # categorize transfers for all but the last transfer
    for idx,deliverer in enumerate(ecy_df['Deliverer']):
        if idx != ecy_df['Deliverer'].shape[0]-1:
            if ((ecy_df['Deliverer'][idx] == ecy_df['Receiver'][idx+1]) &
                (ecy_df['Deliverer'][idx+1] == ecy_df['Receiver'][idx])):
                # count number of cases where there is a delivery both ways
                count += 1
                #two_way = two_way.append(ecy_df.iloc[[idx]])
                try:
                    two_way = pandas.concat([two_way, ecy_df.iloc[[idx]]])
                except:
                    print('here 1')
                    two_way = ecy_df.iloc[[idx]]
                idx_taken = 1
            else:
                if idx_taken:
                    #two_way = two_way.append(ecy_df.iloc[[idx]])
                    try: 
                        two_way = pandas.concat([two_way, ecy_df.iloc[[idx]]])
                    except:
                        print('here 2')
                        two_way = ecy_df.iloc[[idx]]
                    idx_taken = 0
                else:
                    #one_way = one_way.append(ecy_df.iloc[[idx]])
                    try: 
                        one_way = pandas.concat([one_way, ecy_df.iloc[[idx]]])
                    except:
                        print('here 3')
                        one_way = ecy_df.iloc[[idx]]
                    idx_taken = 0
        else:
            # categorize the last entry by comparing with the end - 1 values
            if ((ecy_df['Deliverer'][idx] == ecy_df['Receiver'][idx-1]) &
                (ecy_df['Deliverer'][idx-1] == ecy_df['Receiver'][idx])):
                count += 1
                try:
                    two_way = pandas.concat([two_way, ecy_df.iloc[[idx]]])
                except:
                    print('here 4')
                    two_way = ecy_df.iloc[[idx]]

    return one_way, two_way

## Link vessel names to MMSI with the most AIS ship tracks in 2018, by category
- Vessels ranked in and selected from `binned_mmsi`

In [ ]:
mmsi_names["Smallest Tanker (< 60 m)"]= {
    368009250:"GLOBAL PROVIDER"
}
mmsi_names["Small Tanker (61 - 150 m)"] = {
    355320000:"KIRKEHOLMEN",
    566306000:"MTM HOUSTON",
    477636500:"BOCHEM LONDON"
}
mmsi_names["Handymax (151-200 m)"] = {
    369042000:"AMERICAN FREEDOM",
    367353110:"PELICAN STATE", 
    366855000:"FLORIDA", 
    367134000:"OVERSEAS LOS ANGELES",
    369040000:"AMERICAN ENDURANCE",
    356159000:"OPEC VICTORY",
    212656000:"PANAGIA THALASSINI",
    538006302:"SONGA PRIDE",
    538004682:"ZAO GALAXY",
    538007277:"CHEMBULK LINDY ALICE",
    538004167:"ARGENT IRIS",
    338221000:"OVERSEAS BOSTON",
    477185200:"RAINBOW STAR",
    538005425:"STI BATTERY",
    636017622:"HIGH CHALLENGE",
    538006892:"NAVIG8 SPARK",
    563054800:"NAEBA GALAXY",
    563046100:"HODAKA GALAXY",
    367353070:"EVERGREEN STATE",
    566150000:"MAERSK TIANJIN",
}
mmsi_names["Aframax (201-250 m)"] = {         
    636015178:"AQUALEGACY",
    220518000:"TORM EMILIE",
    255806154:"NORDTULIP",
    311943000:"GULF PEARL",
    538001893:"ELAN VITAL"
}
mmsi_names["Suezmax (251-300 m)"] = {
    303294000:"ALASKAN EXPLORER",
    367688000:"ALASKAN NAVIGATOR", 
    303656000:"ALASKA LEGEND", 
    366948190:"ALASKA FRONTIER"
}

# Count the number of MMSI used in each size class to evaluate cargo capacity using the DOE database
N_MMSI["Smallest"] = len(mmsi_names["Smallest Tanker (< 60 m)"])
N_MMSI["Small"] = len(mmsi_names["Small Tanker (61 - 150 m)"])
N_MMSI["Handymax"] = len(mmsi_names["Handymax (151-200 m)"])
N_MMSI["Aframax"] = len(mmsi_names["Aframax (201-250 m)"])
N_MMSI["Suezmax"] = len(mmsi_names["Suezmax (251-300 m)"])

In [ ]:
for idx,vessel_type in enumerate([*N_MMSI]):
    # get list of vessel names by size class
    vessels = list(mmsi_names[[*mmsi_names][idx]].values())
    volume[vessel_type] = get_ecy_tanker_byvessel(
        vessels, ecy_xls_path, fac_xls_path)
    # sort transfers by transfer date/time
    volume[vessel_type] = volume[vessel_type].sort_values(by='StartDateTime').reset_index()
    # identify consective transfers where Deliverer/Receiver are swapped (two-way, import/export or visa versa)
    one_way[vessel_type], two_way[vessel_type]  = split_ecy_transfers(
        volume[vessel_type])
    print(f'{vessel_type}: {one_way[vessel_type].shape[0]} one-way, {two_way[vessel_type].shape[0]} two-way transfers')

## Create a histogram of tanker volume transfers

In [ ]:
   
fig, ax = plt.subplots(figsize=(10, 5))
bin_width = 5e6
bins = numpy.arange(0,1.7e8,bin_width)
plt.hist(volume[vessel_type]['TransferQtyInLiters'], bins=bins)#, histtype = 'bar', facecolor = 'blue')
bins_labels(bins, fontsize=12)
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2e'))
plt.ylabel("Transfer counts")
#plt.xlabel("Bin Number")
plt.title(f"Histogram of 2018 tanker volume transfers (liters)\n Bin width = {bin_width:1.1e}")
plt.show()

#### Thoughts:
- **Handymax** has the most number of ship tracks in 2018. The 4.5-5e7 liter size class is consistent with Handymax size.  
- The cargo capacities we are using based on SILO are: 
    - **smallest tanker** 5.8e5
    - **small tanker** 1.5e7
    - **handymax**	5.3e7
    - **aframax**	1.26e8
    - **suezmax**	1.85e8
- According to the [ATC](https://www.aktanker.com/fleet-main) website, **Alaskan Explorer's carrying capacity is 206,683,483.4 liters**.  This is true of all their tankers, three of which are the first three in AIS ship track counts in 2018.  
    - 366948190.0   5940
    - 303656000.0  24133
    - 303294000.0  29376
    - 367688000.0  25157
- **Suezmax**:The max transfer is 1.5e8 liters.  Max capacity is 2.1e8.  We classify max capacity of Suezmax as 1.85e8.  
- We use 5 values of oil cargo capacity for tankers based on their size class. The ECY transfers suggest the following max cargo capacities for these categories
    - **Smallest tanker**: 5.8e5 (based on max transfers for MMSI with most ship tracks, also, [Global Provider](https://www.professionalmariner.com/global-provider/) is given a cargo capacity of 567811.768)
    - **Small tanker**: 1.5e7 (based on max transfers for MMSI  with most ship tracks)
    - **Small tanker**: 1.75e7 (based on histogram)
    - **Handymax**: 4e7 (based on max transfers for the two MMSI with most ship tracks)
    - **Handymax**: 4.75e7 (based on histogram)
    - **Aframax**: 1.2e8 (based on histogram)
    - **Suezmax**: 1.5e8 (based on histogram, this is ~71% capacity)
- **Question**: Do we want to adjust any of the values that we are using?  The values I am considering changing are:
    
    -**Handymax** from 5.3e7 to 4.75e7 and 
    -**Suezmax** from 1.85e8 to 1.5e8. 

### Check Small tankers 

In [ ]:
# Get transfers for these two examples of Handymax 
vessels = list(mmsi_names["Small Tanker (61 - 150 m)"].values())
vessel_type = 'Small'
volume[vessel_type] = get_ecy_tanker_byvessel(vessels,ecy_xls_path,fac_xls_path)

# plot histogram of handimax transfers
fig, ax = plt.subplots()
bin_width = 1e7
bins = numpy.arange(0,1.7e8,bin_width)
plt.hist(volume[vessel_type]['TransferQtyInLiters'], bins=bins)#, histtype = 'bar', facecolor = 'blue')
bins_labels(bins, fontsize=12)
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2e'))
plt.ylabel("Transfer counts")
plt.title(f"Histogram of Small tanker volume transfers (liters)")
plt.show()

max(volume[vessel_type]['TransferQtyInLiters'])

### Check Handymax tankers using transfers for the ten vessels with the most AIS ship tracks

In [ ]:
# Get transfers for these two examples of Handymax 
vessels = list(mmsi_names["Handymax (151-200 m)"].values())
vessel_type = 'Handymax'
volume[vessel_type] = get_ecy_tanker_byvessel(vessels,ecy_xls_path,fac_xls_path)

# plot histogram of handimax transfers
fig, ax = plt.subplots()
bin_width = 1e7
bins = numpy.arange(0,1.7e8,bin_width)
plt.hist(volume[vessel_type]['TransferQtyInLiters'], bins=bins)#, histtype = 'bar', facecolor = 'blue')
bins_labels(bins, fontsize=12)
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2e'))
plt.ylabel("Transfer counts")
plt.title(f"Histogram of Handymax tanker volume transfers (liters)")
plt.show()

max(volume['Handymax']['TransferQtyInLiters'])

In [ ]:
vessels = list(mmsi_names["Handymax (151-200 m)"].values())
vessel_type = 'Handymax'
volume[vessel_type] = get_ecy_tanker_byvessel(vessels,ecy_xls_path,fac_xls_path)
pandas.set_option('display.max_rows', volume[vessel_type].shape[0]+1)


# plot histogram of handimax transfers
fig, ax = plt.subplots()
bin_width = 1e7
bins = numpy.arange(0,1.7e8,bin_width)
plt.hist(volume[vessel_type]['TransferQtyInLiters'], bins=bins)#, histtype = 'bar', facecolor = 'blue')
bins_labels(bins, fontsize=12)
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2e'))
plt.ylabel("Transfer counts")
plt.title(f"Histogram of Handymax tanker volume transfers (liters)")
plt.show()



#### Divide transfers into those with both import and export vs. one-way only:
- 100% full tankers that have import and export transfers per terminal stop ("double" transfer) and 
- 50% tankers that have either import or export ("single" transfer) 

In [ ]:
for idx,vessel_type in enumerate([*N_MMSI]):
    # get list of vessel names by size class
    vessels = list(mmsi_names[[*mmsi_names][idx]].values())
    volume[vessel_type] = get_ecy_tanker_byvessel(
        vessels, ecy_xls_path, fac_xls_path)
    volume[vessel_type] = volume[vessel_type].sort_values(by='StartDateTime').reset_index()

In [ ]:
del one_way, two_way
vessel_type="Handymax"
ecy_df = volume[vessel_type].copy()
# one_way=pandas.DataFrame({})
# two_way=pandas.DataFrame({})
count = 0
idx_taken = 0
# order transfers by time
ecy_df = ecy_df.sort_values(by='StartDateTime').reset_index(drop=True)
# categorize transfers for all but the last transfer
for idx,deliverer in enumerate(ecy_df['Deliverer']):
    if idx != ecy_df['Deliverer'].shape[0]-1:
        if ((ecy_df['Deliverer'][idx] == ecy_df['Receiver'][idx+1]) &
            (ecy_df['Deliverer'][idx+1] == ecy_df['Receiver'][idx])):
            # count number of cases where there is a delivery both ways
            count += 1
            #two_way = two_way.append(ecy_df.iloc[[idx]])
            try:
                two_way = pandas.concat([two_way, ecy_df.iloc[[idx]]])
            except:
                two_way = ecy_df.iloc[[idx]]
            idx_taken = 1
        else:
            if idx_taken:
                #two_way = two_way.append(ecy_df.iloc[[idx]])
                try: 
                    two_way = pandas.concat([two_way, ecy_df.iloc[[idx]]])
                except:
                    two_way = ecy_df.iloc[[idx]]
                idx_taken = 0
            else:
                #one_way = one_way.append(ecy_df.iloc[[idx]])
                try: 
                    one_way = pandas.concat([one_way, ecy_df.iloc[[idx]]])
                except:
                    one_way = ecy_df.iloc[[idx]]
                idx_taken = 0
    else:
        # categorize the last entry by comparing with the end - 1 values
        if ((ecy_df['Deliverer'][idx] == ecy_df['Receiver'][idx-1]) &
            (ecy_df['Deliverer'][idx-1] == ecy_df['Receiver'][idx])):
            count += 1
            try:

                two_way = pandas.concat([two_way, ecy_df.iloc[[idx]]])
            except:
                two_way = ecy_df.iloc[[idx]]

In [ ]:
two_way=two_way.reset_index()

In [ ]:
two_way['StartDateTime']=pandas.to_datetime(two_way['StartDateTime'])

In [ ]:
dt_seconds = [(two_way['StartDateTime'][ind+1] - two_way['StartDateTime'][ind]).seconds for ind in numpy.arange(len(two_way)-1)]
dt_days = [(two_way['StartDateTime'][ind+1] - two_way['StartDateTime'][ind]).days for ind in numpy.arange(len(two_way)-1)]
dt_hours = numpy.array(dt_seconds)/3600 + numpy.array(dt_days) * 24

In [ ]:
plt.hist(dt_hours,bins=[0, 24, 48, 72, 96, 120, 144, 168, 700])

### Same length of ECY dataframe when sorted by An=> no duplicate AntIDs.  This is different than what I had seen before, where duplicate AntIDs indicated two-way transfers.  ecy_df.shape

### Repeat of above with all transfers.  This is what I remember seeing from before=> duplicate AntIDs

In [ ]:
ECYdf.shape

In [ ]:
ECYdf.groupby('AntID').count().shape

### Evaluate duplicate AntIDs => multiple oils in one transfer

In [ ]:
ECYdf_antid=ECYdf[['StartDateTime','AntID']].groupby('AntID').count().rename(columns={'StartDateTime':'count'})
ECYdf_antid[ECYdf_antid['count']>1]

In [ ]:
print(f'{vessel_type}: {one_way.shape[0]} one-way, {two_way.shape[0]} two-way transfers')

In [ ]:
#for idx,vessel_type in enumerate([*N_MMSI]):
for idx,vessel_type in enumerate(['Handymax']):
    print(vessel_type)
    one_way[vessel_type], two_way[vessel_type]  = split_ecy_transfers(
        volume[vessel_type])

### Calculate probability of full tanker for handimax

In [ ]:
handyVoyages['full'] = two_way['Handymax'].shape[0]
handyVoyages['half_full'] = 2*one_way['Handymax'].shape[0]
handyVoyages['total'] = handyVoyages['full']+handyVoyages['half_full']
handyFullProbability = (1.0 * handyVoyages['full']/handyVoyages['total'] +
                        0.5 * handyVoyages['half_full']/handyVoyages['total'])
# Assign output to yaml file used in oil_attribution.ipynb
capacity['probability_full_handymax'] = handyFullProbability
print(f'Probability of full Handymax: {handyFullProbability:.2f}')

#### List number of WA state ECY transfers per Handymax
- NOTE: Absent ECY transfers -> CAD traffic

In [ ]:
total = 0
vessels = list(mmsi_names["Handymax (151-200 m)"].values())
for vessel in vessels:
    test = get_ecy_tanker_byvessel(vessel,ecy_xls_path,fac_xls_path)
    print(vessel, test.shape[0])
    total += test.shape[0]
total

#### NOTE: Some of the top tankers in AIS ship tracks are partially or fully CAD and not represented in the ECY transfers

In [ ]:
vessel_type = 'Handymax'
volume[vessel_type]['Deliverer'].shape[0]

In [ ]:
print(f'{two_way[vessel_type].shape[0]} paired transfers')
print(f'{one_way[vessel_type].shape[0]} single transfers')

In [ ]:
ax = one_way[vessel_type]['TransferQtyInLiters'].hist()
ax.set_title('Handymax transfer quantities (liters) for vessels that are 50% full')

In [ ]:
ax = two_way[vessel_type]['TransferQtyInLiters'].hist()
ax.set_title('Handymax transfer quantities (liters) for vessels that are 100% full')

In [ ]:
handimax_combined = one_way[vessel_type].append(two_way[vessel_type])
ax = handimax_combined['TransferQtyInLiters'].hist()
ax.set_title('Handymax transfer quantities (liters) for all vessels')

In [ ]:
max(volume['Handymax']['TransferQtyInLiters'])

In [ ]:
nbins = 15
max_full = max(volume['Handymax']['TransferQtyInLiters'])
# if max(volume['Handymax']['TransferQtyInLiters']) > max_full:
#     raise ValueError('Upper bin limit for Handymax needs to be ajusted up')

bin_values = numpy.arange(
    0,
    max_full + max_full/nbins,
    max_full/nbins
)

fig, ax = plt.subplots(figsize = (3,3))
n, cargo_capacity, patches = ax.hist(
        volume['Handymax']['TransferQtyInLiters'],
        bins = bin_values)
ax.set_xlabel('Transfer volume')
ax.set_ylabel('Number of transfers')
ax.set_ylim(0,30)
ax.set_title(f'Handymax max cargo distribution')

In [ ]:
print(f'Max Handymax volume transfer (liters): {max(volume["Handymax"]["TransferQtyInLiters"])}')

In [ ]:
weights = n/sum(n)
capacity['weights'] = weights.tolist()
#cargo_capacity['amount_in_liters'].pop(0) # Remove zero (lower bound of first bin)
capacity['amount_in_liters'] = cargo_capacity.tolist()
capacity['amount_in_liters'].pop(0)
print(f'{len(capacity["amount_in_liters"])} =? {len(capacity["weights"])}')
capacity

In [ ]:
with open(yaml_out_dir/'handymax_capacity.yaml', 'w') as file:
    documents = yaml.safe_dump(capacity, file)

### Show histograms of Aframax transfers 

In [ ]:
binned_mmsi["Aframax (201-250 m)"].iloc[:5]

In [ ]:
vessels = list(mmsi_names["Aframax (201-250 m)"].values()) 
vessel_type = 'Aframax'
volume[vessel_type] = get_ecy_tanker_byvessel(vessels,ecy_xls_path,fac_xls_path)

# plot histogram of handimax transfers
fig, ax = plt.subplots()
bin_width = 1e7
bins = numpy.arange(0,1.7e8,bin_width)
plt.hist(volume[vessel_type]['TransferQtyInLiters'], bins=bins)#, histtype = 'bar', facecolor = 'blue')
bins_labels(bins, fontsize=12)
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2e'))
plt.ylabel("Transfer counts")
plt.title(f"Histogram of Aframax tanker volume transfers (liters)\n {100*percent_total['Aframax']:.0f}% of Aframax traffic\n Bin width = {bin_width:1.1e}")
plt.show()

max(volume['Aframax']['TransferQtyInLiters'])

### Take away:  In some cases, multiple AntIDs are used to represent cargo transfers for a given voyage.  The sum of these is close to 1e8, which approximates SILO value of 1.26e8

### Show histograms of Suezmax transfers 

In [ ]:
# vessels = ['ALASKAN EXPLORER', "ALASKAN NAVIGATOR", "ALASKA LEGEND", "ALASKA FRONTIER"] 
# vessel_type = 'Suezmax'
# volume[vessel_type] = get_ecy_tanker_byvessel(vessels,ecy_xls_path,fac_xls_path)

# fig, ax = plt.subplots()
# bin_width = 1e7
# bins = numpy.arange(0,1.7e8,bin_width)
# plt.hist(volume[vessel_type]['TransferQtyInLiters'], bins=bins)#, histtype = 'bar', facecolor = 'blue')
# bins_labels(bins, fontsize=12)
# ax.xaxis.set_major_formatter(FormatStrFormatter('%.2e'))
# plt.ylabel("Transfer counts")
# plt.title(f"Histogram of Suezmax tanker volume transfers (liters)\n {100*percent_total['Suezmax']:.0f}% traffic, Bin width = {bin_width:1.1e}")
# plt.show()

# max(volume['Suezmax']['TransferQtyInLiters'])

# Cargo transfer evaluations 

### Handymax

In [ ]:
# vessels = ["AMERICAN FREEDOM","PELICAN STATE", "FLORIDA",
#           'PANAGIA THALASSINI','CHEMBULK LINDY ALICE','ZAO GALAXY',
#            'OVERSEAS LOS ANGELES','AMERICAN ENDURANCE','OPEC VICTORY','SONGA PRIDE',] 
# vessel_type = 'Handymax'

# ### Get a sense of behavior with fueling
# volume[vessel_type] = ECYdf.loc[
#     (ECYdf.Deliverer.isin(vessels) |
#      ECYdf.Receiver.isin(vessels)),
#     ['TransferQtyInGallon', 'Deliverer','Receiver','StartDateTime','AntID']
# ].groupby('AntID').agg(
#     {'TransferQtyInGallon':'sum','Deliverer':'first', 'Receiver':'first','StartDateTime':'first'}
#     ).sort_values(by='TransferQtyInGallon',ascending=False)

# volume[vessel_type]['TransferQtyInLiters'] = gal2liter*volume[vessel_type]['TransferQtyInGallon']
# volume[vessel_type]=volume[vessel_type].sort_values(by='StartDateTime')
# pandas.set_option('display.max_rows', volume[vessel_type].shape[0]+1)
# volume[vessel_type]

### Aframax

In [ ]:
### Get a sense of behavior with fueling
# volume['Aframax'] = ECYdf.loc[
#     (ECYdf.Deliverer.isin(["AQUALEGACY","TORM EMILIE","NORDTULIP","GULF PEARL","ELAN VITAL"] ) |
#      ECYdf.Receiver.isin(["AQUALEGACY","TORM EMILIE","NORDTULIP","GULF PEARL","ELAN VITAL"] )),
#     ['TransferQtyInGallon', 'Deliverer','Receiver','StartDateTime','AntID']
# ].groupby('AntID').agg(
#     {'TransferQtyInGallon':'sum','Deliverer':'first', 'Receiver':'first','StartDateTime':'first'}
#     ).sort_values(by='TransferQtyInGallon',ascending=False)

# volume['Aframax']['TransferQtyInLiters'] = gal2liter*volume['Aframax']['TransferQtyInGallon']
# volume['Aframax']=volume['Aframax'].sort_values(by='StartDateTime')
# pandas.set_option('display.max_rows', volume['Aframax'].shape[0]+1)
# volume['Aframax']

### Suezmax

#### Take-away:  Indeed...big boats distribute in smaller transfers.  Show transfers to evaluate.

In [ ]:
# volume['Suezmax']=volume['Suezmax'].sort_values(by='StartDateTime')
# pandas.set_option('display.max_rows', volume['Suezmax'].shape[0]+1)
# volume['Suezmax']

In [ ]:
### Get a sense of behavior with fueling
# volume['Suezmax'] = ECYdf.loc[
#     (ECYdf.Deliverer.isin(['ALASKAN EXPLORER', "ALASKAN NAVIGATOR", "ALASKA LEGEND", "ALASKA FRONTIER"]) |
#      ECYdf.Receiver.isin(['ALASKAN EXPLORER', "ALASKAN NAVIGATOR", "ALASKA LEGEND", "ALASKA FRONTIER"])),
#     ['TransferQtyInGallon', 'Deliverer','Receiver','StartDateTime','AntID']
# ].groupby('AntID').agg(
#     {'TransferQtyInGallon':'sum','Deliverer':'first', 'Receiver':'first','StartDateTime':'first'}
#     ).sort_values(by='TransferQtyInGallon',ascending=False)

# volume['Suezmax']['TransferQtyInLiters'] = gal2liter*volume['Suezmax']['TransferQtyInGallon']
# volume['Suezmax']=volume['Suezmax'].sort_values(by='StartDateTime')
# pandas.set_option('display.max_rows', volume['Suezmax'].shape[0]+1)
# volume['Suezmax']

## Aframax

In [ ]:
### Get a sense of behavior with fueling
# volume['Suezmax'] = ECYdf.loc[
#     (ECYdf.Deliverer.isin(['ALASKAN EXPLORER', "ALASKAN NAVIGATOR", "ALASKA LEGEND", "ALASKA FRONTIER"]) |
#      ECYdf.Receiver.isin(['ALASKAN EXPLORER', "ALASKAN NAVIGATOR", "ALASKA LEGEND", "ALASKA FRONTIER"])),
#     ['TransferQtyInGallon', 'Deliverer','Receiver','StartDateTime','AntID']
# ].groupby('AntID').agg(
#     {'TransferQtyInGallon':'sum','Deliverer':'first', 'Receiver':'first','StartDateTime':'first'}
#     ).sort_values(by='TransferQtyInGallon',ascending=False)

# volume['Suezmax']['TransferQtyInLiters'] = gal2liter*volume['Suezmax']['TransferQtyInGallon']
# volume['Suezmax']=volume['Suezmax'].sort_values(by='StartDateTime')
# pandas.set_option('display.max_rows', volume['Suezmax'].shape[0]+1)
# volume['Suezmax']